In [1]:
# Some reference is taken from here: https://captum.ai/tutorials/Bert_SQUAD_Interpret

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import json

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

from transformers import BertModel, BertTokenizer

from sklearn.preprocessing import LabelEncoder

from captum.attr import LayerIntegratedGradients
from captum.attr import visualization as viz

import time
from tqdm import tqdm

import pickle

from utils import *

/home/fahim/anaconda3/envs/explainable-news/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-10 17:49:46.122795: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 17:49:46.538209: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/usr/local/cuda-11.7/lib64
2023-03-10 17:49:46.538260: W tensorflow/compiler/xla/stream_executor/

In [2]:
label_enc = LabelEncoder()

with open(f'label_encoder.pkl', 'rb')as f:
    
    label_enc = pickle.load(f)
    
    f.close()

/home/fahim/anaconda3/envs/explainable-news/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
label_enc.classes_

array(['bangladesh', 'economy', 'education', 'entertainment',
       'international', 'life-style', 'opinion', 'sports', 'technology'],
      dtype=object)

In [4]:
bert_model_name = "csebuetnlp/banglabert_large"
bert = BertModel.from_pretrained(bert_model_name)
tokenizer = BertTokenizer.from_pretrained(bert_model_name)

device = 'cpu'

model = ClassifyNews(bert)

saved_model_path = f'saved_weights/1678450980.0084934-news_large.pth'

checkpoint = torch.load(saved_model_path)

model.load_state_dict(checkpoint)

model.to(device)

model.eval()

You are using a model of type electra to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at csebuetnlp/banglabert_large were not used when initializing BertModel: ['electra.encoder.layer.7.attention.self.value.bias', 'electra.encoder.layer.19.output.dense.weight', 'electra.encoder.layer.14.intermediate.dense.weight', 'electra.encoder.layer.13.attention.self.key.weight', 'electra.encoder.layer.7.intermediate.dense.bias', 'electra.encoder.layer.12.attention.self.value.weight', 'electra.encoder.layer.22.attention.self.query.weight', 'electra.encoder.layer.11.intermediate.dense.weight', 'electra.encoder.layer.9.intermediate.dense.weight', 'electra.encoder.layer.5.attention.self.query.weight', 'electra.encoder.layer.19.attention.self.key.bias', 'electra.encoder.layer.11.output.LayerNorm.weight', 'electra.encoder.layer.19.output.LayerNorm.bias', 'electra.encoder.layer.22.output.LayerNorm.bias'

ClassifyNews(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise

In [56]:
data = pd.read_csv(f'data/cleaned_data.csv')

_, test_data = train_test_split(data, test_size = 0.2, random_state = 2023, stratify = data['label'])

In [57]:
test_data_file = NewsDatasets(test_data)

test_loader = torch.utils.data.DataLoader(test_data_file, batch_size=1, shuffle=True)

In [58]:
real_data = next(iter(test_loader))

In [59]:
text, labels = real_data

In [60]:
len(text)

1

In [61]:
labels.shape

torch.Size([1])

In [62]:
labels

tensor([4])

In [63]:
text[0]

'৩০ সন্ত্রাসীর মৃত্যুর আদেশে পাকিস্তানের সেনাপ্রধানের স্বাক্ষর[SEP]পাকিস্তানে ৩০ সন্ত্রাসীর মৃত্যুর আদেশে স্বাক্ষর দেশটির সেনাপ্রধান জেনারেল কামার জাভেদ এসব সন্ত্রাসীর কয়েকজন ২০১৪ সালে পেশোয়ারের সেনা স্কুলে হামলায় অভিযুক্ত হামলায় স্কুলের ১৪৪ শিক্ষার্থী নিহত পেশোয়ারের সেনা স্কুলে জঙ্গি হামলার ২০১৫ সালের জানুয়ারিতে সামরিক আদালত স্থাপন আদালতে বড় শাস্তি প্রদানের সেনাপ্রধানের অনুমোদন প্রয়োজন পাকিস্তানের আন্তবাহিনী জনসংযোগ পরিদপ্তর সূত্র জানায় এসব জঙ্গি সৈন্যদের অপহরণের জবাই বিমানবন্দরে হামলা বেসামরিক নাগরিকদের হত্যা পাকিস্তানি সশস্ত্র বাহিনী নিরাপত্তা বাহিনীর ওপর হামলার যুক্ত'

In [64]:
label_enc.inverse_transform(labels.numpy())

array(['international'], dtype=object)

## Create input and baseline for explainability

In [65]:
def input_and_baseline(text):
    
    tokenizer_config = {
        "max_length": 100,
        "truncation": True,
        "add_special_tokens": False
    }
    
    baseline_token_id = tokenizer.pad_token_id 
    sep_token_id = tokenizer.sep_token_id 
    cls_token_id = tokenizer.cls_token_id 

    text_ids = tokenizer.encode(text, **tokenizer_config)
    
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
   
    raw_tokens = tokenizer.convert_ids_to_tokens(input_ids)
  

    baseline_input_ids = [cls_token_id] + [baseline_token_id] * len(text_ids) + [sep_token_id]
    
    return torch.tensor([input_ids], device = 'cpu'), torch.tensor([baseline_input_ids], device = 'cpu'), raw_tokens

In [66]:
input_ids, baseline_input_ids, all_tokens = input_and_baseline(text[0])

In [67]:
input_ids

tensor([[    2,  2415,  7481,   411,  3163, 17957,  2927, 21384,   764,  7363,
             3,  6305,  2415,  7481,   411,  3163, 17957,  7363,  6654, 21384,
          3763, 13980, 22516,  1613,  7481,   411,     1,  5172,  1460,     1,
          4365,  3606,     1,  5678,     1,  4114, 17318,  5082,  2846,     1,
          4365,  3606,  6031,  5108,  5335,  1780,     1,  3617,  3517,  5427,
          3905,     1,  4085, 10250, 21384,   764,  6507,     1,  2927,  2308,
          7377, 18663,   936, 30070,   411,  2708,     1,  1613,  6031, 13764,
         20015, 18254, 11222,  3794, 11384,  9832,  2562,  4960,  7995,  3592,
          3808,  3854,  1615,  5108,  1833,     3]])

In [79]:
baseline_input_ids

tensor([[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]])

In [80]:
print(" ".join(all_tokens))

[CLS] ৩০ সন্ত্রাসী ##র মৃত্যুর আদেশে পাকিস্তানের সেনাপ্রধান ##ের স্বাক্ষর [SEP] পাকিস্তানে ৩০ সন্ত্রাসী ##র মৃত্যুর আদেশে স্বাক্ষর দেশটির সেনাপ্রধান জেনারেল কামার জাভেদ এসব সন্ত্রাসী ##র [UNK] ২০১৪ সালে [UNK] সেনা স্কুলে [UNK] অভিযুক্ত [UNK] স্কুলের ১৪৪ শিক্ষার্থী নিহত [UNK] সেনা স্কুলে জঙ্গি হামলার ২০১৫ সালের [UNK] সামরিক আদালত স্থাপন আদালতে [UNK] শাস্তি প্রদানের সেনাপ্রধান ##ের অনুমোদন [UNK] পাকিস্তানের আন্ত ##বাহিনী জনসংযোগ পরি ##দপ্ত ##র সূত্র [UNK] এসব জঙ্গি সৈন্যদের অপহরণের জবাই বিমানবন্দরে হামলা বেসামরিক নাগরিকদের হত্যা পাকিস্তানি সশস্ত্র বাহিনী নিরাপত্তা বাহিনীর ওপর হামলার যুক্ত [SEP]


In [81]:
def model_output(inputs):
    return model(inputs)[0]

In [82]:
lig = LayerIntegratedGradients(model_output, model.bert.embeddings)

In [83]:
attributions, delta = lig.attribute(inputs= input_ids.to(device),
                                    baselines= baseline_input_ids.to(device),
                                    n_steps = 500,
                                    return_convergence_delta=True
                                    )

In [84]:
print(attributions.size())

torch.Size([1, 86, 1024])


In [85]:
torch.argmax(model(input_ids)[0]).numpy()

array(4)

In [86]:
torch.max(model(input_ids)[0])

tensor(4.8337, grad_fn=<MaxBackward1>)

In [87]:
model(input_ids)

tensor([[ 1.5820,  0.0872, -0.7721, -0.0417,  4.8337, -0.3294,  1.7847, -0.9160,
         -0.2395]], grad_fn=<AddmmBackward0>)

In [88]:
def summarize_attributions(attributions):

    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()
    
    return attributions

attributions_sum = summarize_attributions(attributions)

In [89]:
score_vis = viz.VisualizationDataRecord(
                        word_attributions = attributions_sum,
                        pred_prob = torch.max(model(input_ids)[0]),
                        pred_class = torch.argmax(model(input_ids)[0]).numpy(),
                        true_class = labels.item(),
                        attr_class = text,
                        attr_score = attributions_sum.sum(),       
                        raw_input_ids = all_tokens,
                        convergence_score = delta)

viz.visualize_text([score_vis])